In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split

In [3]:
# Path to MedMNIST dataset directory
data_dir = '/kaggle/input/kvasir-dataset/kvasir-dataset'

# Load dataset
img_size = (224, 224)
batch_size = 128

dataset = image_dataset_from_directory(
    directory=data_dir,
    labels='inferred',
    label_mode='categorical',  # Corrected from 'cateorical' to 'categorical'
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

# Get class names before applying map()
class_names = dataset.class_names
num_classes = len(class_names)

# Normalize pixel values to [0, 1]
def normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(normalize_image)

# Split into training and testing datasets
cardinality = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.8 * cardinality)
test_size = cardinality - train_size

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Print the number of classes
print("Number of classes:", num_classes)


Found 4000 files belonging to 8 classes.
Number of classes: 8


In [9]:
import tensorflow as tf

def build_VggNet():
    model = tf.keras.Sequential()
    
    # Block 1
    model.add(tf.keras.layers.Conv2D(input_shape=(224, 224, 3), filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 5
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Fully Connected Layers (to match VGG16's default top layers)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dense(1000, activation='softmax'))  # Output layer for 1000 classes in ImageNet
    
    return model

# Instantiate the model
model = build_VggNet()

# Load pre-trained weights
weights_path = '/kaggle/input/vgg_16_weights/tensorflow2/default/1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
model.load_weights(weights_path)

# Modify top layers for your specific task
model.pop()  # Remove the default output layer
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

# Freeze all layers except the new top layers
for layer in model.layers[:-3]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)                   │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_33 (Conv2D)                   │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_35 (Conv2D)                   │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_36 (Conv2D)                   │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_37 (Conv2D)                   │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_38 (Conv2D)                   │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 135,377,224 (516.42 MB)

 Trainable params: 1,116,680 (4.26 MB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [11]:
history = model.fit(
    train_dataset,
    steps_per_epoch=100,
    validation_data=test_dataset,
    validation_steps=10,
    epochs=100
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 224ms/step - accuracy: 0.2294 - loss: 2.1586 - val_accuracy: 0.3187 - val_loss: 1.6574
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 223ms/step - accuracy: 0.4486 - loss: 1.4119 - val_accuracy: 0.6175 - val_loss: 1.1146
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 218ms/step - accuracy: 0.5577 - loss: 1.0812 - val_accuracy: 0.6263 - val_loss: 0.9733
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.6305 - loss: 0.9364 - val_accuracy: 0.6737 - val_loss: 0.8193
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.6545 - loss: 0.8462 - val_accuracy: 0.6413 - val_loss: 0.8254
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 219ms/step - accuracy: 0.6922 - loss: 0.7686 - val_accuracy: 0.6812 - val_loss: 0.7665
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.6938 - loss: 0.7409 - val_accuracy: 0.7387 - val_loss: 0.6905
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.7131 -

In [12]:
# Evaluate the model
loss, acc = model.evaluate(test_dataset)
print('Test Accuracy:', acc)

7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 344ms/step - accuracy: 0.7871 - loss: 0.4579
Test Accuracy: 0.7900000214576721


In [13]:
# Save the model to a file
model.save('VggNet_16_kvasir_accurate.h5')

In [19]:
from PIL import Image
import numpy as np

def predict_single_image(model, image_path, class_names, target_size=(224, 224)):
    """
    Predicts the class of a single image.
    
    Parameters:
    - model: Trained TensorFlow/Keras model.
    - image_path: Path to the input image.
    - class_names: List of class names.
    - target_size: Tuple, the target size of the image (default: (28, 28)).
    
    Returns:
    - Predicted class name.
    """
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")  # Convert to RGB
    image = image.resize(target_size)  # Resize to target dimensions
    image_array = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Predict the class
    predictions = model.predict(image_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class]

    return predicted_class_name

# Example usage
image_path = '/kaggle/input/kvasir-dataset/kvasir-dataset/ulcerative-colitis/15d46b18-10b4-4bb8-b7b8-1d56c1349b8d.jpg'  # Double the backslashes or use forward slashes
predicted_class = predict_single_image(model, image_path, class_names)
print(f"Predicted Class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: ulcerative-colitis
